## Coverage for the Poisson Confidence Intervals

In this example we wnat to estimate the coverage for different methods used to estimate the confidence interval in case of a Poisson process. 
Our esperiment observes *n* events when $\mu$ events are expected. 
The probability distribution for the *n* observed events is the Poisson distribution:

 $$P (n | \mu ) = \mu^n \frac { e ^{-\mu} } { n !}$$
 
We estimate the coverage for two possible methods to estimate the interval

1.  Normal approximation. The interval is computed simply using  \sqrt(n) 
2.  Frequentist Poisson intervals (Garwood intervals). These are exact frequentist intervals computed with central ordering. 

To compute the coverage we scan the Poisson parameter $\mu$, the number of expected events in a given range and using a pre-defined number of points. 
For each value of the parameter $\mu$ we look at each observed value *n* and at its probbility given by the Poisson distribution. 
FOr each $n$ observed an interval can be computed and we can check if the corresponding value of $\mu$ is inside or outside of the interval. The coverage is computed from the totsal probabilities of having $\mu$ inside the interval. 

In [1]:
%jsroot on

In [2]:
mu_min = 0.5;
mu_max = 6.5;
nscan_points= 300;

#### 1. Normal Approximation 

To compute the coverage, we need to define a function checking if given a value of $n$ event observed the parameter $\mu$ is inside or outside the interval. 

We create first such function for the normal intervals, computed using $\pm \sqrt{n}$


In [3]:
auto IsInside1 = [](double nobs, double mu) { 
    double xmin = nobs - sqrt(nobs);
    double xmax = nobs + sqrt(nobs);
    return ( mu > xmin && mu <= xmax); 
  };

#### 2. Garwood Interval 

Now we create the function for the exact frequentist Poisson interval (Garwood intervals). 
See for example https://en.wikipedia.org/wiki/Poisson_distribution#Confidence_interval.

Note that these interval are provided in the **TH1** class in ROOT by calling first the function
`TH1::SetBinErrorOption(TH1::kPoisson)` and then `TH1::GetBinErrorLow(ibin)` with `TH1::GetBinErrorUp(ibin)`.
   

In [4]:
auto IsInside2 = [](double nobs, double mu) { 
    // using frequentist Poisson intervals
    double alpha = 1.- 0.682689492;
    double xmin = (nobs >0) ? ROOT::Math::gamma_quantile( alpha/2, nobs, 1.) : 0;
    double xmax = ROOT::Math::gamma_quantile_c( alpha/2, nobs+1, 1);
    return ( mu > xmin && mu <= xmax); 
  };

#### Computation of the Coverage

We create first an histogram for all the values of the Poisson parameter $\mu$ for which we compute the coverage.

In [5]:
h1 = new TH1D("h1","Coverage for Poisson measurement;mu;Coverage probability",nscan_points,mu_min,mu_max);

We now loop on different parameter values of $\mu$. For each parameter value we observe a value $n$, with a probability given by the Poisson distribution. 

For each possible $n$ (i.e. values which have a probability significantly different than zero), we compute an interval and we check if interval includes the given parameter value. We compute then the integrated probability for all observed values, which give an interval that has the corresponding parameter $\mu$ inside. 

The coverage corresponds to this computed probability and should be as close as possible to the confidence level used to compute the interval.
In this example we have decided to compute the interval using the $1 \sigma$ confidence level, i.e. `0.682689492`.

To change the interval estimation method,  we just need to change the name of the function used to compute the interval and to check if the parameter is inside or outside

In [6]:
h1->Reset();
for (int i = 1; i <= h1->GetNbinsX(); ++i) {
    double mu = h1->GetXaxis()->GetBinCenter(i);
    //printf("%d %f \n",i,mu);
    double probInside = 0;
    for (int nobs = 0; nobs < 100; ++nobs) { 
        // compute the interval using the given rule
        bool inside = IsInside1(nobs,mu);   // using sqrt(n)
        //bool inside = IsInside2(nobs,mu);  // using Garwood intervals
        if (inside) {
            double prob = ROOT::Math::poisson_pdf(nobs,mu);
            probInside += prob; 
            // break the loop - no need to get to prob too small
            //if (probInside > 1.E-10 && prob < 1.E-10) break;
        } 
    } 
    if (i %10 == 0) printf("i = %d probability for mu = %f - prob = %f \n",i,mu,probInside); 
    h1->SetBinContent(i,probInside);
}

i = 10 probability for mu = 0.690000 - prob = 0.465488 
i = 20 probability for mu = 0.890000 - prob = 0.528124 
i = 30 probability for mu = 1.090000 - prob = 0.566205 
i = 40 probability for mu = 1.290000 - prob = 0.682625 
i = 50 probability for mu = 1.490000 - prob = 0.710234 
i = 60 probability for mu = 1.690000 - prob = 0.723781 
i = 70 probability for mu = 1.890000 - prob = 0.725335 
i = 80 probability for mu = 2.090000 - prob = 0.556668 
i = 90 probability for mu = 2.290000 - prob = 0.584247 
i = 100 probability for mu = 2.490000 - prob = 0.603154 
i = 110 probability for mu = 2.690000 - prob = 0.613912 
i = 120 probability for mu = 2.890000 - prob = 0.710572 
i = 130 probability for mu = 3.090000 - prob = 0.720635 
i = 140 probability for mu = 3.290000 - prob = 0.724258 
i = 150 probability for mu = 3.490000 - prob = 0.536232 
i = 160 probability for mu = 3.690000 - prob = 0.631940 
i = 170 probability for mu = 3.890000 - prob = 0.645801 
i = 180 probability for mu = 4.090000 - 

We plot now the computed coverage as function of $\mu$

In [7]:
h1->SetMinimum(0.3);
h1->Draw("HIST  ");
//h1->SetLineColor(kRed); h1->Draw("HIST SAME");}
//nominal confidence level for interval
level = 0.682689492;  // = 2.*ROOT::Math::normal_cdf_c(1,1);
l = new TLine(h1->GetXaxis()->GetXmin(),0.682689492,h1->GetXaxis()->GetXmax(),0.682689492);
l->SetLineStyle(kDashed);
l->Draw();
gPad->Draw();
firstPlot = false;

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


As a last example we put here the computation of the interval in case of a likelihood 

In [8]:
TF1 poissonNll("f","x-[nobs]*log(x)");
auto IsInside3 = [&](int nobs, double mu) { 
    // compute the nll interval for mu   
    double muHat = nobs; // best value of mu
    poissonNll.SetParameter(0,nobs);
    double nllmin = poissonNll(muHat);
    double xmin = poissonNll.GetX(nllmin+0.5,0,nobs-0.01);
    double xmax = poissonNll.GetX(nllmin+0.5,nobs+0.01,2*nobs+2);
    return ( mu > xmin && mu <= xmax); 
};